In [1]:
from drn_interactions.io import load_waveforms
from drn_interactions.config import Config
from typing import List
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import minmax_scale
import seaborn as sns


from drn_interactions.spiketrains.waveforms import WaveformPreprocessor, WaveformOutliers, WaveformPeaks, WaveMap
sns.set_theme(context="talk", style="ticks")


%load_ext autoreload
%autoreload 2

In [2]:
df = load_waveforms()[["neuron_id", "waveform_index", "waveform_value"]]

In [3]:
# preprocess waveforms using median and gaussian filters then scale to [0, 1]

preprocessor = WaveformPreprocessor(
    gaussian_sigma=2, 
    medfilter_size=9,
    scale_to_baseline=False,
    minmax=True,
    )
df_preprocessed = preprocessor(df)

In [4]:
# remove bad waveforms using an isolation forest

outlier_mod = WaveformOutliers()
df_good, df_outliers = outlier_mod(df_preprocessed)

In [5]:
# find peaks and calculate their widths and prominences using scipy

peak_finder = WaveformPeaks(width_rel_height=0.8, base_rel_height=0.8)
df_summary = df_good.apply(peak_finder, axis=1, summarize=True,).reset_index()
df_summary = df_summary.loc[lambda x: ~np.isnan(x.prom_min)]
df_summary["prom_basepost"] = df_summary["prom_basepost"].fillna(0)
# df_summary["width_min"] = np.where(df_summary["width_min"] < 25, np.nan, df_summary["width_min"])

In [6]:
# calculate waveform embeddings using UMAP

decomposer = WaveMap(n_neighbors=20, n_components=2)
df_decomposed = decomposer(df_good).reset_index()

In [7]:
# save results

df_summary.to_csv(Config.derived_data_dir / "waveform_summary.csv", index=False)
df_decomposed.to_csv(Config.derived_data_dir / "wavemap.csv", index=False)